In [78]:
import pandas as pd
import re
import translators.server as tss
TARGET_LANG = 'es'

In [79]:
# read the latest mass production table
# for 7.17
# path_new = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new_cleaned.txt"
# path_ref = './Greg_table_folder/7.17 - Sheet1.csv'
# path_output = './Output_translated_folder/7.17 HTML/7.17 HTML/MassProduction/7.17_translated.txt'
# path_no_mark = './Output_translated_folder/7.17 HTML/7.17 HTML/MassProduction/7.17_translated_no_mark.txt'

# for 7.10
# path_new = "./Output_cleaned_folder/7.10 HTML/7.10 HTML/MassProduction/7_10_cleaned.txt"
# path_ref = './Greg_table_folder/7.10 - Sheet1.csv'
# path_output = './Output_translated_folder/7.10 HTML/7.10 HTML/MassProduction/7.10_translated.txt'
# path_no_mark = './Output_translated_folder/7.10 HTML/7.10 HTML/MassProduction/7.10_translated_no_mark.txt'

# for 6.10
# path_new = "./Output_cleaned_folder/6.10 HTML/6.10 HTML/MassProduction/finalMassProdtable_cleaned.txt"
# path_ref = './Greg_table_folder/6.10 Translation - Sheet1.csv' # input optional
# path_output = './Output_translated_folder/6.10 HTML/6.10 HTML/MassProduction/6.10_translated.txt' # output
# path_no_mark = './Output_translated_folder/6.10 HTML/6.10 HTML/MassProduction/6.10_translated_no_mark.txt' # output

# for 6.05
path_new = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.txt"
path_ref = './Greg_table_folder/6.05 - Sheet1.csv'
path_output = './Output_translated_folder/6.05 HTML/6.05 HTML/MassProduction/6.05_translated.txt'
path_no_mark = './Output_translated_folder/6.05 HTML/6.05 HTML/MassProduction/6.05_translated_no_mark.txt'

In [80]:
table_clean = pd.read_csv(path_new, sep="\t", index_col=0, keep_default_na=False)

In [81]:
# Change column name
table_clean_col_list = []
for i in table_clean:
    table_clean_col_list.append(i)
    table_clean_col_list.append(i+"ESP")
len(table_clean_col_list)

# Create a new table, with the column from the Greg's mass production table and the index from the latest mass production table
table_translated = pd.DataFrame(columns=table_clean_col_list, index=table_clean.index)
table_translated_clean = pd.DataFrame(columns=table_clean_col_list, index=table_clean.index)
table_translated

,1,1ESP,2,2ESP,3,3ESP,4,4ESP,5,5ESP,6,6ESP,7,7ESP,8,8ESP
Problem Name,,,,,,,,,,,,,,,,
%(startStateNodeName)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(statement)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(graphic)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(question1)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(question2)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
%(Input_74_percent4)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(Input_78_simplifytar1tot110)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(Input_82_simplifytar2tot210)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# read the Greg's mass production table
table_old = pd.read_csv(path_ref, header = None)
# table_old.head()
# skip some rows, because they are translation for HTML elements
header_index = table_old.index[table_old[0] == 'Problem Name'].to_list()
header_index
# reload the csv
table_old = pd.read_csv(path_ref, header = header_index)
table_old.columns
# table_old = None

Index(['Problem Name', 'Problem1', 'ProblemESP1', 'Problem Name.1', 'Problem2',
       'ProblemESP2', 'Problem Name.2', 'Problem3', 'ProblemESP3',
       'Problem Name.3', 'Problem4', 'ProblemESP4', 'Problem Name.4',
       'Problem5', 'ProblemESP5', 'Problem Name.5', 'Problem6', 'ProblemESP6',
       'Problem Name.6', 'Problem7', 'ProblemESP7', 'Problem Name.7',
       'Problem8', 'ProblemESP8'],
      dtype='object')

In [83]:
def replace_variable(content_new, column, row):
    pattern_variable = "%\((.*?)\)%"
    # count the number of the replacement in one variable(content_new)
    match_variable = [match for match in re.findall(pattern_variable, str(content_new))]
    for i in range(len(match_variable)):
        # find the variable
        variable = "%(" + str(match_variable[i]) + ")%"
        # find the corresponding column name, and then find the value
        column_name = table_clean.columns[column]
        try:
            value = table_clean.loc[variable, column_name] 
            # print(variable, value)
            content_new = content_new.replace(variable, value)
            table_clean.iloc[row, column] = content_new
        except:
            print(column_name, variable + " doesn't exist")
    return content_new

In [84]:
def replace_string(formula, column, row):
    pattern_string = '"(.*?)"'
    match_string = [match for match in re.findall(pattern_string, formula)]
    # print("old", formula)
    if match_string:
        for i in range(len(match_string)):
            string = str(match_string[i])
            formula = formula.replace(string, replace_variable(string, column, row)) 
    # print("new", formula)  
    return formula

In [85]:
def translate_string(formula):
    pattern_string = '"(.*?)"'
    match_string = [match for match in re.findall(pattern_string, formula)]
    # print("old", formula)
    if match_string:
        for i in range(len(match_string)):
            string = str(match_string[i])
            if string == '':
                pass
            else:
                try:
                    translation = tss.google(string, from_language='en', to_language=TARGET_LANG)
                except:
                    translation = 'error'
                formula = formula.replace(string, translation) 
    # print("new", formula)  
    return formula

In [86]:
def replace_formula(content_new, column=None, row=None, is_translate=False, replacement="#"):
    replacement_sign = replacement
    replacement_formula_dic = {}
    pattern_formula = "<%(.*?)%>"
    # count the number of the replacement in one variable(content_new)
    match_formula = [match for match in re.findall(pattern_formula, str(content_new))]
    if match_formula:
        for i in range(len(match_formula)):
            old_formula = match_formula[i]
            if '"' in old_formula:
                if is_translate:
                    formula = translate_string(old_formula)
                else:
                    formula = replace_string(old_formula, column, row)
                final_formula = str(formula)
            else:
                final_formula = str(old_formula)
            replacement = replacement_sign + str(i)
            replacement_formula_dic[replacement] = final_formula
            content_new = content_new.replace(old_formula, replacement)
    return content_new, replacement_formula_dic

In [87]:
# find the translation of the latest mass production table from the Greg's mass production table 
google_count = 0
google_dict = {}
for column in range(table_clean.shape[1]):
    for row in range(table_clean.shape[0]):
        # get the location
        column_name = table_clean.columns[column]
        column_num = table_translated.columns.get_loc(column_name)
        content_new = str(table_clean.iloc[row, column])
        if table_clean.index[row] == '%(startStateNodeName)%':
            print("skip '%(startStateNodeName)%'")
            # write the english column
            table_translated[column_name].iloc[row] = content_new
            table_translated_clean[column_name].iloc[row] = content_new
            content_translated = content_new
            content_translated_clean = content_new
        # digit or empty, keep the original
        # elif content_new.lstrip('-').replace(".", "").isdigit() or content_new == '':
        elif re.match(r"^(?!-0?(\.0+)?$)-?(0|[1-9]\d*)?(\.\d+)?(?<=\d)$", content_new) or content_new == '':
            print("digital or empty")
            table_translated[column_name].iloc[row] = content_new
            table_translated_clean[column_name].iloc[row] = content_new
            content_translated = content_new
            content_translated_clean = content_new
        # replace & translate
        else:
            print(content_new)
            # replace
            # judge whether a formula inside the content new
            pattern_formula = "<%(.*?)%>"
            # count the number of the replacement in one variable(content_new)
            match_formula = [match for match in re.findall(pattern_formula, str(content_new))]
            if match_formula:
                content_new, replacement_formula_dic = replace_formula(content_new, column, row)
            else:
                replacement_formula_dic = None
            # print("before", content_new)
            content_new = replace_variable(content_new, column, row)
            # print("after", content_new)
            if bool(replacement_formula_dic) is False:
                pass
            else:
                for key, value in replacement_formula_dic.items():
                    content_new = content_new.replace(key, value)
            table_translated[column_name].iloc[row] = content_new
            table_translated_clean[column_name].iloc[row] = content_new
            # translate
            # search in dict
            if content_new in google_dict:
                print("find translation in dict")
                content_translated = '[google]' + google_dict[content_new]
                content_translated_clean = google_dict[content_new]
            # translate in google
            elif table_old is None or table_old.columns[(table_old == content_new).any()].empty:
                print("use google translation")
                # print(content_new)
                if match_formula:
                    # content_new, translation_formula_dic = translate_formula(content_new)
                    content_new, translation_formula_dic = replace_formula(content_new, is_translate=True)
                else:
                    translation_formula_dic = None
                try:
                    translation = tss.google(content_new, from_language='en', to_language=TARGET_LANG)
                except:
                    translation = 'error'
                # print(content_new, translation_formula_dic)
                # print(translation)
                if bool(translation_formula_dic) is False:
                    pass
                else:
                    for key, value in translation_formula_dic.items():
                        content_new = content_new.replace(key, value)
                        translation = translation.replace(key, value)
                content_translated = '[google]' + translation
                content_translated_clean = translation
                google_dict[content_new] = translation 
            # find translation in old table
            else:
                print("find translation in sheet")
                column_name_old = table_old.columns[(table_old == content_new).any()][0]
                column_num_old = table_old.columns.get_loc(column_name_old)
                content_translated = table_old[table_old[column_name_old] == content_new].iloc[0, column_num_old+1]
                content_translated_clean = table_old[table_old[column_name_old] == content_new].iloc[0, column_num_old+1]    
        print(content_translated)
        table_translated.iloc[row, column_num+1] = content_translated
        table_translated_clean.iloc[row, column_num+1] = content_translated_clean
# table_translated

skip '%(startStateNodeName)%'
example1
Using the set of shapes in the picture, answer questions 1 through 4 in the worksheet provided.
find translation in sheet
Usando el conjunto de formas en la imagen, responda las preguntas 1 a 4 en la hoja de trabajo provista.
Assets/01.png
find translation in sheet
Assets/01.png
(1) What percent of the shapes are %(type1)%?
find translation in sheet
(1) ¿Qué porcentaje de las formas son cuadrados?
(2) What percent of the shapes are %(type2)%?
find translation in sheet
(2) ¿Qué porcentaje de las formas son moradas?
(3) What percent of the shapes are %(type3)%?
find translation in sheet
(3) ¿Qué porcentaje de las formas no son azules?
(4) What percent of the shapes are %(type4)%?
find translation in sheet
(4) ¿Qué porcentaje de las formas son rojas?
squares
use google translation
[google]cuadrícula
purple
use google translation
[google]púrpura
not blue
use google translation
[google]no azul
red
use google translation
[google]roja rojo
digital or emp

In [88]:
print(google_count)

0


In [89]:
# export the csv
table_translated_output = table_translated
# table_translated_output = table_translated.fillna("[CTAT]NAN")
table_translated_output.to_csv(path_output, encoding="utf-8", sep="\t")
table_translated_clean.to_csv(path_no_mark, encoding="utf-8", sep="\t")